In [79]:
import sys
from pathlib import Path
from IPython.display import display, Audio

import torch
from torch import optim
from pytorch_lightning import Trainer

import stempeg
import librosa
from scipy.io.wavfile import write

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
sys.path.append("../../")
import asteroid_filterbanks
import asteroid

In [ ]:
from asteroid.asteroid.models import ConvTasNet
from asteroid.asteroid.losses import pairwise_neg_sisdr, PITLossWrapper

from asteroid.asteroid.data import MUSDB18Dataset

from asteroid.asteroid.engine import System

In [77]:
DATA_DIR = Path("../../data")
SAMPLE_RATE = 44100

In [83]:
EX_FILE = str(list((DATA_DIR/"train").glob("*.mp4"))[0])

In [86]:
tracks, sr = stempeg.read_stems(EX_FILE)

In [91]:
len(tracks)
for i in range(len(tracks)):
    name = f"/Users/Alexandre/Desktop/{i}.wav"
    stempeg.write_audio(name, tracks[i], sr)

In [74]:
stempeg.Info(EX_FILE).duration(1)

171.24

In [138]:
%autoreload 2

train_loader = MUSDB18Dataset(
    root=str(DATA_DIR),
    targets=["vocals", "bass", "drums", "other"],
    suffix=".mp4",
    split="train",
    subset=None,
    segment=3,
    samples_per_track=1,
    random_segments=True,
    random_track_mix=False,
    sample_rate=SAMPLE_RATE
)

val_loader = MUSDB18Dataset(
    root=str(DATA_DIR),
    targets=["vocals", "bass", "drums", "other"],
    suffix=".mp4",
    split="test",
    subset=None,
    segment=3,
    samples_per_track=1,
    random_segments=True,
    random_track_mix=False,
    sample_rate=SAMPLE_RATE
)

3it [00:00, 25.41it/s]
3it [00:00, 40.35it/s]


In [139]:
model = ConvTasNet(n_src=4, sample_rate=SAMPLE_RATE)

loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [154]:
def train(model, dataset, criterion, optim, epochs):
    for epoch in range(epochs):
        epoch_loss = 0
        for train_batch in dataset:
            optim.zero_grad()

            x, y = train_batch
            if len(x.size()) == 1 and len(y.size()) == 2:
                batch_size = 1
                x = x[None, :]
                y = y[None, :]
                
            n_batch, n_sources, n_samples = y.size()
            y = y.reshape(n_batch, n_sources, n_samples)
            
            output = model(x)

            loss = criterion(output, y)
            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch} - Loss: {epoch_loss}")

In [155]:
train(model, train_loader, loss, optimizer, 1)

In [ ]:
path = "/content/drive/MyDrive/IA321/data/musdb18/prep/test/AM Contra - Heart Peripheral/mixture.wav"
original = librosa.load(path, sr=SAMPLE_RATE)
output = model.separate(path)